# Object References, Mutability, and Recycling

## Variables are not boxes

In [1]:
a = [1, 2, 3]
b = a
a.append(4)
b

[1, 2, 3, 4]

In [3]:
d = 5
c = 5

In [4]:
d == c

True

In [5]:
d is c

True

In [6]:
id(d), id(c)

(140693923168624, 140693923168624)

In [7]:
class Gizmo:
    def __init__(self):
        print(f'Gizmo id: {id(self)}')

x = Gizmo()  #1

Gizmo id: 140693877191392


In [9]:
y = Gizmo() * 10  #2

Gizmo id: 140693319602704


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [10]:
dir()

['Gizmo',
 'In',
 'Out',
 '_',
 '_1',
 '_4',
 '_5',
 '_6',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__vsc_ipynb_file__',
 '_dh',
 '_i',
 '_i1',
 '_i10',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_i7',
 '_i8',
 '_i9',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'a',
 'b',
 'c',
 'd',
 'exit',
 'get_ipython',
 'open',
 'quit',
 'x']

## Identity, Equality, and Aliases

In [11]:
charles = {'name': 'Charles L. Dogson', 'born': 1832}
lewis = charles  #1
lewis is charles

True

In [12]:
id(charles), id(lewis)  #2

(140693318767296, 140693318767296)

In [13]:
lewis['balance'] = 950  #3
charles

{'name': 'Charles L. Dogson', 'born': 1832, 'balance': 950}

In [14]:
alex = {'name': 'Charles L. Dogson', 'born': 1832, 'balance': 950}
alex == charles  #1

True

In [16]:
alex is not charles  #2

True

In [17]:
id(alex), id(charles)

(140693877720064, 140693318767296)

### Choosing Between == and is

In [18]:
# The == operator compares the values of objects (the data they hold),
# while 'is' compares their identies

### The Relative Immutability of Tuples

In [1]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
t1 == t2

True

In [2]:
id(t1[-1])

140098052991552

In [4]:
t1[-1].append(99)
t1

(1, 2, [30, 40, 99])

In [5]:
id(t1[-1])

140098052991552

In [6]:
t1 == t2

False

## Copies Are Shallow by Default

In [7]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)  #1 the easiest way to make a copy
l2

[3, [55, 44], (7, 8, 9)]

In [8]:
l2 == l1

True

In [9]:
l2 is l1

False

In [10]:
id(l2[-1]), id(l1[-1])

(140097654230016, 140097654230016)

In [11]:
# other way to make a copy
l2 = l1[:]
l2

[3, [55, 44], (7, 8, 9)]

Using the constructor or _[:]_ produces a _shallow copy_ (i.e., the outermost container is duplicated,
but the copy is filled with references to the same items held by the original container). This saves memory and causes no problem if all the items are immutable. But if there are mutable items, this may lead to unpleasant surprises.

In [12]:
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)  #1
l1.append(100)  #2
l1[1].remove(55)  #3
print('l1:', l1)
print('l2:', l2)
l2[1] += [33, 22]  #4
l2[2] += (10, 11)
print('l1:', l1)
print('l2:', l2)

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]
l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


### Deep and Shallow Copies of Arbitrary Objects

In [1]:
class Bus:
    def __init__(self, passengers=None) -> None:
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)

In [2]:
import copy

bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)

id(bus1), id(bus2), id(bus3)  #1

(140066904488784, 140066903297472, 140066903294688)

In [3]:
bus1.drop('Bill')
bus2.passengers  #2

['Alice', 'Claire', 'David']

In [5]:
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)  #3

(140066903793344, 140066903793344, 140066903651008)

In [6]:
bus3.passengers  #4

['Alice', 'Bill', 'Claire', 'David']

In [7]:
# cyclic references

a = [10, 20]
b = [a, 30]
a.append(b)
a

[10, 20, [[...], 30]]

In [10]:
from copy import deepcopy
c = deepcopy(a)
c

[10, 20, [[...], 30]]

## Function Parameters as References

In [1]:
def f(a, b):
    a += b
    return a

x = 1
y = 2
f(x, y)

3

In [2]:
x, y  #1

(1, 2)

In [3]:
a = [1, 2]
b = [3, 4]
f(a, b)

[1, 2, 3, 4]

In [4]:
a, b  #2

([1, 2, 3, 4], [3, 4])

In [5]:
t = (10, 20)
u = (30, 40)
f(t, u)

(10, 20, 30, 40)

In [6]:
t, u  #3

((10, 20), (30, 40))

### Mutable Types as Parameter Defaults: Bad Idea

In [3]:
class HaunteBus:
    """A bus model haunted by ghost passengers"""

    def __init__(self, passengers=[]):  #1
        self.passengers = passengers  #2

    def pick(self, name):
        self.passengers.append(name)  #3

    def drop(self, name):
        self.passengers.remove(name)

In [4]:
bus1 = HaunteBus(['Alice', 'Bill'])  #1
bus1.passengers

['Alice', 'Bill']

In [5]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers  #2

['Bill', 'Charlie']

In [6]:
bus2 = HaunteBus()  #3
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [7]:
bus3 = HaunteBus()  #4
bus3.passengers  #5

['Carrie']

In [8]:
bus3.pick('Dave')
bus2.passengers  #6

['Carrie', 'Dave']

In [9]:
bus2.passengers is bus3.passengers  #7

True

In [10]:
bus1.passengers  #8

['Bill', 'Charlie']

In [11]:
dir(HaunteBus.__init__)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [12]:
HaunteBus.__init__.__defaults__

(['Carrie', 'Dave'],)

In [13]:
HaunteBus.__init__.__defaults__[0] is bus2.passengers

True

### Defensive Programming with Mutable Parameters